Никитина Ирина (team3)

In [ ]:
import re # Регулярные выражения.
import pymorphy2 # Морфологический анализатор.
from collections import Counter # Не считать же частоты самим.
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('train.csv', sep='\t', encoding='utf8')

In [3]:
def delete_tags(text):
    findtags = re.compile("</?\w+>", re.S)
    text = "".join(findtags.split(text))
    return text

In [4]:
# из описания удаляем html-тэги
df['new'] = df['description'].apply(func = delete_tags)

In [5]:
morph=pymorphy2.MorphAnalyzer()
def getMeaningfullWords(text):
    words=[]
    tokens=re.findall('[А-Яа-яЁёA-Za-z]+\-[А-Яа-яЁёA-Za-z]+|[А-Яа-яЁёA-Za-z]+', text)
    for t in tokens:
        pv=morph.parse(t)
        for p in pv:
            if p.tag.POS in ['ADJF', 'NOUN', 'VERB']:
                words.append(p.normal_form)
                break
    return words
def getMeaningfullString(txt):
    res = ' '.join(getMeaningfullWords(txt))
    return res

In [93]:
fields=[]     # Все поля фичи
dictionaries=[] # Посчитанные словари (векторное представление фич).
morph=pymorphy2.MorphAnalyzer()
needPos = False

posConv={'ADJF':'_ADJ','NOUN':'_NOUN','VERB':'_VERB'}
def getFeatureDictionary(text, needPos=None):
    words=[a[0] for a in re.findall("([А-ЯЁа-яёA-Za-z]+(-[А-ЯЁа-яёA-Za-z]+)*)", text)]
    reswords=[]
    
    for w in words:
        wordform=morph.parse(w)[0]
        try:
            if wordform.tag.POS in ['ADJF', 'NOUN', 'VERB']:
                if needPos!=None:
                    reswords.append(wordform.normal_form+posConv[wordform.tag.POS])
                else:
                    reswords.append(wordform.normal_form)
        except:
            pass
            
    stat=Counter(reswords)
#        stat={a: stat[a] for a in stat.keys() if stat[a]>1}
    return stat

    # Посчитаем вектора для всех статей.
def calcFeatureDictionaries(needPos=None):
        for a in fields:
            dictionaries.append(getFeatureDictionary(a, needPos))

def cosineSimilarity(a, b):
    if len(a.keys())==0 or len(b.keys())==0:
        return 0
    sumab=sum([a[na]*b[na] for na in a.keys() if na in b.keys()])
    suma2=sum([a[na]*a[na] for na in a.keys()])
    sumb2=sum([b[nb]*b[nb] for nb in b.keys()])
    return sumab/math.sqrt(suma2*sumb2)

In [29]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cross_validation import train_test_split
counter12=CountVectorizer(ngram_range=(1,2))

/usr/local/lib/python3.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [228]:
X = counter12.fit_transform(df['name'].values)
y = df['target']

logreg = sklinear.LogisticRegression(C=1, penalty='l2')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

model = logreg.fit(X_train, y_train)
errors=np.zeros((2,2))
prediction = model.predict(X_test)
for i in np.arange(0, len(prediction)):
    errors[prediction[i]][y_test.values[i]] += 1
print(errors)

[[30307.  1073.]
 [ 1535. 27085.]]


In [31]:
import sklearn.linear_model as sklinear
from sklearn.linear_model import LogisticRegressionCV
from sklearn.cross_validation import KFold

In [32]:
fold = KFold(len(y_train), n_folds=5, shuffle=True, random_state=777)

In [174]:
searchCV = LogisticRegressionCV(
        Cs=list(np.power(10.0, np.arange(-10, 10)))
        ,penalty='l2'
        ,scoring='roc_auc'
        ,cv=fold
        ,random_state=777
        ,max_iter=10000
        ,fit_intercept=True
        ,solver='newton-cg'
        ,tol=1e-4
        ,class_weight='balanced'
    )
searchCV.fit(X_train, y_train)

print ('Max auc_roc:', searchCV.scores_[1].mean(axis=0).max())

/usr/local/lib/python3.7/site-packages/scipy/optimize/linesearch.py:422: LineSearchWarning: Rounding errors prevent the line search from converging
  warn(msg, LineSearchWarning)
/usr/local/lib/python3.7/site-packages/scipy/optimize/linesearch.py:313: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/usr/local/lib/python3.7/site-packages/sklearn/utils/optimize.py:195: UserWarning: Line Search failed
  warnings.warn('Line Search failed')


Max auc_roc: 0.9875428122953179


In [184]:
feature_list = counter12.get_feature_names()
count12_test = CountVectorizer(ngram_range=(1,2), vocabulary = feature_list)
df_test = pd.read_csv('test.csv', sep='\t', encoding='utf8')
X_test_test = count12_test.transform(df_test['name'].values)
final_doc = pd.DataFrame()
final_doc['id'] = df_test['id'] 
final_doc['target'] = searchCV.predict_proba(X_test_test)[:, 1]
final_doc.to_csv('subm1.csv', index=False)

# Второй вариант - сокращаем число фич

In [211]:
df['new_name'] = df['name'].apply(func = getMeaningfullString)
df['new_name'] = df['new_name'].apply(lambda x: x.replace('секунда', ''))
counter12_2 = CountVectorizer(ngram_range=(1,2))
X2 = counter12_2.fit_transform(df['new_name'].values)
y2 = df['target']

X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, test_size=0.3, random_state=42)

In [219]:
fold2 = KFold(len(y_train2), n_folds=5, shuffle=True, random_state=777)
searchCV2 = LogisticRegressionCV(
        Cs=list(np.power(10.0, np.arange(-10, 10)))
        ,penalty='l2'
        ,scoring='roc_auc'
        ,cv=fold2
        ,random_state=777
        ,max_iter=10000
        ,fit_intercept=True
        ,solver='newton-cg'
        ,tol=1e-4
        ,class_weight='balanced'
    )
searchCV2.fit(X_train2, y_train2)

print ('Max auc_roc:', searchCV2.scores_[1].mean(axis=0).max())

/usr/local/lib/python3.7/site-packages/scipy/optimize/linesearch.py:313: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/usr/local/lib/python3.7/site-packages/sklearn/utils/optimize.py:195: UserWarning: Line Search failed
  warnings.warn('Line Search failed')


Max auc_roc: 0.9849883980107391


In [ ]:
feature_list2 = counter12_2.get_feature_names()
count12_test2 = CountVectorizer(ngram_range=(1,2), vocabulary = feature_list2)
df_test['new_name'] = df_test['name'].apply(func = getMeaningfullString)
df_test['new_name'] = df_test['new_name'].apply(lambda x: x.replace('секунда', ''))

X_test_test2 = count12_test2.transform(df_test['new_name'].values)
final_doc2 = pd.DataFrame()
final_doc2['id'] = df_test['id'] 
final_doc2['target'] = searchCV2.predict_proba(X_test_test2)[:, 1]
final_doc2.to_csv('subm2.csv', index=False)

# подключаем описание

In [222]:
df.head()

,id,name,description,target,new,new_name
0,0,Заведующий отделом/секцией в магазин YORK (Уру...,<p><strong>В НОВЫЙ МАГАЗИН YORK (хозтовары) пр...,1,В НОВЫЙ МАГАЗИН YORK (хозтовары) приглашаем на...,отдел секция век магазин уручей
1,1,Наладчик станков и манипуляторов с ПУ,Обязанности:работа на токарных станках с ЧПУ T...,0,Обязанности:работа на токарных станках с ЧПУ T...,наладчик станок исполняющий манипулятор
2,2,Разработчик С++ (Криптограф),<strong>Требования:</strong> <ul> <li>Опыт про...,0,Требования: Опыт программирования на С++ Знан...,разработчик криптограф
3,3,Фрезеровщик,<p>Условия:</p> <ul> <li>На работу вахтовым ме...,0,Условия: На работу вахтовым методом в г. Калу...,фрезеровщик
4,4,Мерчендайзер/продавец-консультант,<p><strong>Компания Палладиум Стандарт - призн...,1,Компания Палладиум Стандарт - признанный лидер...,мерчендайзер продавец-консультант


In [22]:
df['new_descr'] = df['new'].apply(getMeaningfullString)

In [23]:
df.to_csv('df_new.csv')

In [24]:
df.head()

,id,name,description,target,new,new_descr
0,0,Заведующий отделом/секцией в магазин YORK (Уру...,<p><strong>В НОВЫЙ МАГАЗИН YORK (хозтовары) пр...,1,В НОВЫЙ МАГАЗИН YORK (хозтовары) приглашаем на...,век новый магазин хозтовары приглашать постоян...
1,1,Наладчик станков и манипуляторов с ПУ,Обязанности:работа на токарных станках с ЧПУ T...,0,Обязанности:работа на токарных станках с ЧПУ T...,обязанность работа токарный станок секунда ток...
2,2,Разработчик С++ (Криптограф),<strong>Требования:</strong> <ul> <li>Опыт про...,0,Требования: Опыт программирования на С++ Знан...,требование опыт программирование секунда знани...
3,3,Фрезеровщик,<p>Условия:</p> <ul> <li>На работу вахтовым ме...,0,Условия: На работу вахтовым методом в г. Калу...,условие работа вахтовый метод век год калуга т...
4,4,Мерчендайзер/продавец-консультант,<p><strong>Компания Палладиум Стандарт - призн...,1,Компания Палладиум Стандарт - признанный лидер...,компания палладиум стандарт признанный лидер п...


In [25]:
df['new_descr'] = df['new_descr'].apply(lambda x: x.replace('век', ''))


In [27]:
df['new_descr'] = df['new_descr'].apply(lambda x: x.replace('секунда', ''))

In [30]:
counter1 = CountVectorizer(ngram_range=(1,1))
X = counter1.fit_transform(df['new_descr'].values)
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [33]:
logreg = sklinear.LogisticRegression(C=1, penalty='l2')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

model = logreg.fit(X_train, y_train)
errors=np.zeros((2,2))
prediction = model.predict(X_test)
for i in np.arange(0, len(prediction)):
    errors[prediction[i]][y_test.values[i]] += 1
print(errors)

[[30366.  1483.]
 [ 1476. 26675.]]


In [34]:
counter12 = CountVectorizer(ngram_range=(1,2))
X_name = counter12.fit_transform(df['name'].values)

In [93]:
from scipy.sparse import hstack

X_all = hstack((X_name, X))

In [94]:
X_all.shape

(200000, 168753)

In [76]:
X_train, X_test, y_train, y_test = train_test_split(X_all, y, test_size=0.3, random_state=42)

In [77]:
model = logreg.fit(X_train, y_train)
errors=np.zeros((2,2))
prediction = model.predict(X_test)
for i in np.arange(0, len(prediction)):
    errors[prediction[i]][y_test.values[i]] += 1
print(errors)

[[30622.   957.]
 [ 1220. 27201.]]


In [82]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve

In [100]:
c_range = np.arange(0.1, 0.5, 0.1)
for C in c_range:
    model = sklinear.LogisticRegression(penalty='l2', fit_intercept=True, C=C)
    model.fit(X_train, y_train)

    s = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])
    print(C, ':\t', s)

KeyboardInterrupt: 

In [102]:
roc_auc_score(y, model.predict_proba(X_all)[:, 1])

0.995470229993192

In [86]:
df_test = pd.read_csv('test.csv', sep='\t', encoding='utf8')
df_test['new_descr'] = df['description'].apply(getMeaningfullString)


In [87]:
df_test.to_csv('df_test_new.csv')

In [96]:
features1 = counter1.get_feature_names()
features12 = counter12.get_feature_names()
counter12_test = CountVectorizer(ngram_range=(1,2), vocabulary=features12)
counter1_test = CountVectorizer(ngram_range=(1,1), vocabulary=features1)

X_test_descr = counter1_test.transform(df_test['new_descr'].values)
X_test_name = counter12_test.transform(df_test['name'].values)
X_all_test = hstack((X_test_name, X_test_descr))
X_all_test.shape

(170179, 168753)

In [97]:
df_test.head()

,id,name,description,new_descr
0,200000,Дизайнер-консультант мебели,<p><strong>Обязанности:</strong></p> <ul> <li>...,век новый магазин хозтовары приглашать постоян...
1,200001,Продавец-консультант (ТЦ на Пушкина),<p><strong>Обязанности</strong>:</p> <p>∙ конс...,обязанность работа токарный станок секунда ток...
2,200002,Менеджер по продажам,<p>Торговый Дом «Форт» это ведущая компания Пе...,требование опыт программирование секунда знани...
3,200003,Продавец-консультант в магазин одежды (ТЦ Волн...,<p><strong>Требуются продавцы консультанты в м...,условие работа вахтовый метод век год калуга т...
4,200004,Специалист по охране труда,<strong>Обязанности:</strong> <ul> <li> <p>осу...,компания палладиум стандарт признанный лидер п...


In [98]:
model = sklinear.LogisticRegression(penalty='l2', fit_intercept=True, C=0.3)
model.fit(X_train, y_train)

final_doc = pd.DataFrame()
final_doc['id'] = df_test['id'] 
final_doc['target'] = model.predict_proba(X_all_test)[:, 1]
final_doc.to_csv('subm3.csv', index=False)